In [1]:
# import necessary packages
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt

In [2]:
class QuantumRegister:
    
    # initialize column vector
    def __init__(self, N):
    
        # define number of qubits
        self.N = N
        
        # sets all states to equal probability of the 0 state with amplitude 1/sqrt(2^N)
        self.psi = 1 / np.sqrt(2**self.N) * np.zeros((2**N, 1))
        
        # set the amplitude of the first state 
        self.psi[0] = 1
    
    def GetRegister(self):
        
        # method to return the quantum state
        return self.psi
    
    def GetProbabilities(self):
        
        # measure probability of each basis state
        return np.abs(self.psi)**2

    def MeasureState(self):
        
        # measure probability of each basis state
        probabilities = np.abs(self.psi)**2
        
        # select a basis state randomly according to the probabilities
        measurement = np.random.choice(np.arange(len(self.psi)), p = probabilities.ravel())
        
        # returns the basis state
        return "| " + np.binary_repr(measurement, width = self.N) + " >"

    def TargetState(self):
        
        # sets the target state so that all digits are 1,1,1...1,0
        target = [1] * (self.N - 1)
        target.append(0)
        
        # converts the array to a string for ease of printing
        targetPrint = ''.join(str(x) for x in target)
        
        # returns the target state
        return "| " + targetPrint + " >"

    def ApplyGate(self, gate):
        
        # performs matrix multiplication of two arrays
        self.psi = np.matmul(gate, self.psi)

In [3]:
class Hadamard:  
    
    @staticmethod
    def HadamardTransform(N):
        
        # define the initial Hadamard matrix
        hadamard_initial = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])
        
        # number of iterations for the transformation
        num_iterations = N - 1
        
        # initialize the resulting tensor product matrix
        hadamard_result = hadamard_initial
        
        # perform tensor product iteratively
        for i in range(num_iterations):
            hadamard_result = np.kron(hadamard_initial, hadamard_result)

        # returns the transformed Hadamard array
        return hadamard_result
    
    @staticmethod
    def ApplyHadamard(QuantReg):
        
        # sets N to same value as established previously
        N = QuantReg.N

        # performs the Hadamard transform
        HadamardGate = Hadamard.HadamardTransform(N)

        # applies the Hadamard to the basis state
        QuantReg.ApplyGate(HadamardGate)

In [4]:
class Oracle:
    
    @staticmethod
    def OracleGate(N, target):

        # creates an identity matrix of 1's on the diagonal
        oracleInitalise = np.eye(2**N)

        # initialise oracle matrix with target
        oracleInitalise[target - 1][target - 1] = -1

        # returns oracle matrix
        return oracleInitalise
    
    @staticmethod
    def ApplyOracle(QuantReg):
        
        # sets N to same value as established previously
        N = QuantReg.N

        # sets the targeted element of the array
        target = (2**N) - 1

        # creates the oracle matrix with targeted element
        OracleOperator = Oracle.OracleGate(N, target)

        # applies the oracle to the state
        QuantReg.ApplyGate(OracleOperator)

In [5]:
class Diffusion:
    
    @staticmethod
    def DiffusionOperator(N):
        
        # defines the superposition state
        s = np.zeros((2**N, 1))

        # sets the first element in the array to 1
        s[0][0] = 1
        
        # computes the conjugate transpose of the s state  
        ss_dagger = np.dot(s, s.T.conj())
        
        # creates an identity matrix
        identity = np.eye(2**N)
        
        # performs the diffusion operator by doubling the conjugate transpose minues the identity matrix
        diffusion =  (2 * ss_dagger) - identity 

        # returns the result of the diffusion operator
        return diffusion
    
    @staticmethod
    def ApplyDiffusion(QuantReg):

        # sets N to same value as established previously
        N = QuantReg.N

        # creates the diffusion operator
        DiffusionGate = Diffusion.DiffusionOperator(N)

        # performs the diffusion operator on the state
        QuantReg.ApplyGate(DiffusionGate)

In [6]:
class Grover:
    @staticmethod
    
    def ApplyAlgorithm(QuantReg):

        # sets N to same value as established previously
        N = QuantReg.N

        # creates an array of probabilities to be used for plotting
        probs = []

        # creates the y-axis of the plot
        states = list(range(1, 1 + (2**N)))

        # performs an amplification for the ideal number of times based on N
        IdealLoop = int((np.pi / 4) * np.sqrt(2**N))

        # performs the first Hadamard transform
        Hadamard.ApplyHadamard(QuantReg)

        # loop that applies the oracle, Hadamard, diffusion operator, and Hadamard to the state
        for i in range(IdealLoop):
            Oracle.ApplyOracle(QuantReg)
            Hadamard.ApplyHadamard(QuantReg)
            Diffusion.ApplyDiffusion(QuantReg)
            Hadamard.ApplyHadamard(QuantReg)
            
            #final_answer = Hadamard.ApplyHadamard(QuantReg)
            #final_answer = np.abs(final_answer)**2
            #probs.append(final_answer)
            
            #plt.plot(states, final_answer, label = [i + 1])
            #plt.xticks(states)
            #plt.xlabel("States")
            #plt.ylabel("Probability")
            #plt.legend()
            #plt.xticks(states)

In [7]:
""" Runs Grover's Search Algorithm """

# defines the number of qubits
qr = QuantumRegister(4)

print("The starting state:\n", qr.MeasureState())
print("\nThe starting quantum register:\n", qr.GetProbabilities())

# runs grover's search algorithm
Grover.ApplyAlgorithm(qr)

print("\nThe ending state:\n", qr.MeasureState())
print("\nThe ending quantum register:\n", qr.GetProbabilities())

#plt.legend(title = "Number of Grover Iteration")
#plt.grid()
#plt.show()

The starting state:
 | 0000 >

The starting quantum register:
 [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

The ending state:
 | 1110 >

The ending quantum register:
 [[0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.00257874]
 [0.96131897]
 [0.00257874]]
